In [7]:
# Keras와 TensorFlow를 사용한 영화 리뷰 텍스트 분류
import os
import sys
from google.colab import drive

# 드라이브 마운트 및 패키지 경로 작업
# drive.mount("/content/drive", force_remount=True)
# my_path = "/content/package"
# save_path = "/content/drive/MyDrive/Colab Notebooks/package"        # 패키지가 저장될 경로

# os.symlink(save_path, my_path)
# sys.path.insert(0, my_path)

# 패키지 설치
# !pip install --target=$my_path tensorflow-hub
# !pip install --target=$my_path tensorflow-datasets

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# print("Version: ", tf.__version__)
# print("Eager mode: ", tf.executing_eagerly())
# print("Hub version: ", hub.__version__)
# print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

# IMDB 데이터셋 다운로드
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

# 데이터 탐색
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
# train_examples_batch
# train_labels_batch

# 모델 구성
## 문장을 임베딩시키기 위해 tensorflow hub 모델을 사용하는 keras층 생성
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"          # 사전 훈련된 텍스트 임베딩 모델
hub_layer = hub.KerasLayer(embedding,
                           input_shape=[],
                           dtype=tf.string,
                           trainable=True)
# 입력 텍스트 길이에 상관없이 임베딩의 출력 크기는 (num_examples, embedding_dimension)
# hub_layer(train_examples_batch[:3])

## 모델 생성
model = tf.keras.Sequential([
    hub_layer,
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)])

# model.summary()

## 손실 함수와 옵티마이저
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),    # 확률을 다루는데 적합
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

# 모델 평가
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

Epoch 1/10
30/30 [==============================] - 49s 2s/step - loss: 0.6589 - accuracy: 0.5268 - val_loss: 0.6198 - val_accuracy: 0.5843
Epoch 2/10
30/30 [==============================] - 47s 2s/step - loss: 0.5614 - accuracy: 0.6974 - val_loss: 0.5150 - val_accuracy: 0.7371
Epoch 3/10
30/30 [==============================] - 48s 2s/step - loss: 0.4312 - accuracy: 0.8184 - val_loss: 0.4181 - val_accuracy: 0.8208
Epoch 4/10
30/30 [==============================] - 46s 2s/step - loss: 0.3190 - accuracy: 0.8785 - val_loss: 0.3558 - val_accuracy: 0.8494
Epoch 5/10
30/30 [==============================] - 47s 2s/step - loss: 0.2328 - accuracy: 0.9135 - val_loss: 0.3213 - val_accuracy: 0.8525
Epoch 6/10
30/30 [==============================] - 47s 2s/step - loss: 0.1706 - accuracy: 0.9441 - val_loss: 0.3072 - val_accuracy: 0.8665
Epoch 7/10
30/30 [==============================] - 47s 2s/step - loss: 0.1248 - accuracy: 0.9619 - val_loss: 0.3053 - val_accuracy: 0.8702
Epoch 8/10
30/30 [==